In [20]:
# import dependencies 
from splinter import Browser 
from bs4 import BeautifulSoup as soup

import pandas as pd

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path':'chromedriver.exe'}
browser = Browser('chrome', **executable_path)

### Article and title 

In [3]:
# Visit the mars nasa news site
# tell splinter to  visit the qoutes to scrape site 
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page ( to respect terms of service and site servers)
# searching for specific combination of tags and attributes ex -- ul.item_list
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
# html parser 
# parse all of the html of the page with beautiful soup
html = browser.html
news_soup = soup(html, 'html.parser')
# "."" is used for selecting classes, pinpoints the <li /> tag with the class of slide and the <ul /> tag with a class of item_list.
# will select last item 
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
# look inside of slide_elem for data within div tag that hass cotent_title as class
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8812/5-hidden-gems-are-riding-aboard-nasas-perseverance-rover/" target="_self">5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover</a></div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover"

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

"The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative."

### Featured Images

In [14]:
# Visit the webpage.
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

# # Find and click the "full image" button # using the unique "id=full_image"
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

# Find the "more info" button and click 
# button doesnt have any unique identifiers ( eg. class or id), so searching for it by text
# checks if element is present and returns bool 
browser.is_element_not_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

# Parse the resulting html with soup 
html = browser.html
img_soup = soup(html,'html.parser')

# pull the most recent image via tags and not direct image link (so that we get the updated image every time and not the specified same image)
# Find the relative image url
#Note src link is in double qoutes("") and not single('')

img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

True

'/spaceimages/images/largesize/PIA18358_hires.jpg'

In [18]:
# image only uses partial link, which is its location within the site. to get full link simply append partial onto base link of site 
base_url = 'https://www.jpl.nasa.gov'
img_url = f'{base_url}{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18358_hires.jpg'

### facts

In [33]:
# read html page with pandas
#pandas specifically searches for and returns a list of tables found
df = pd.read_html('https://space-facts.com/mars/')
df

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [38]:
#select mars planet profile table 
df[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [39]:
df = df[0].copy()

In [41]:
# rename columns 
df.columns=['description', 'value']
# set description column as index 
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [42]:
# convert extracted table to html so it can be displayed in web app/webpage
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [43]:
# close automated browser session
browser.quit()